In [1]:
import _pickle as pkl
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import shapely.geometry as geo
import numpy as np
import networkx as nx
import pandas as pd
from scipy.optimize import minimize

In [2]:
# File paths
# devens_map_file = '/home/sai/TopometricRegistration/devensData/devens_map/devens_map_poly.pkl'
osm_nodes_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_nodes.txt'
osm_edges_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_edges.txt'
osm_nodes_gt_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_nodes_truth.txt'
osm_edges_gt_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_edges_truth.txt'
# cloud_file = '/home/sai/Downloads/scans_in_utm_annotated_small.pkl'
plot_dir = '/home/sai/TopometricRegistration/cache/'
full_scan_file = '/home/sai/Downloads/scans_in_utm_annotated.pkl'


In [ ]:
# Load in data
# devens_map = pkl.load(open(devens_map_file, 'rb'), encoding = 'bytes')
osm_nodes = np.genfromtxt(osm_nodes_file, delimiter = ',')
osm_edges = np.genfromtxt(osm_edges_file, delimiter = ',', dtype = int)
osm_nodes_gt = np.genfromtxt(osm_nodes_gt_file, delimiter = ',')
osm_edges_gt = np.genfromtxt(osm_edges_gt_file, delimiter = ',', dtype = int)

# cloud = pd.read_pickle(cloud_file)
full_scan = pd.read_pickle(full_scan_file)

In [ ]:
print(scan_active.shape)
print(osm_nodes_active.shape) 
print(osm_nodes_gt_active.shape)
print(osm_nodes_active)
print(osm_nodes_gt_active)


In [ ]:
def transform_devens_2d(x):
    osm_new = np.zeros((osm_nodes_active.shape[0], 2))
    for i in range(osm_nodes_active.shape[0]):
        x_new = np.cos(x[0]) * osm_nodes_active[i,0] + np.sin(x[0]) * osm_nodes_active[i,1] + x[1]
        y_new = -np.sin(x[0]) * osm_nodes_active[i,0] + np.cos(x[0]) * osm_nodes_active[i,1] + x[2]

        osm_new[i,0] = x_new
        osm_new[i,1] = y_new
    return osm_new

In [ ]:
def find_closest(x_new, y_new):
    closest = 100000.0
    for j in scan_active.shape[0]:
        dist = np.sqrt((x_new - scan_active[j,0])**2 + (y_new - scan_active[j,1])**2)
        if dist == 0:
            return dist
        if dist < closest:
            closest = dist

    return closest

In [ ]:

def cost_func_2d(x):
    my_cost = 0
    for i in range(osm_nodes_active.shape[0]//100):
        dx = np.random.normal(0, 1)
        dy = np.random.normal(0, 1)
        x_noise = osm_nodes_active[i*100,0] + dx
        y_noise = osm_nodes_active[i*100,1] + dy
        x_new = np.cos(x[0]) * x_noise + np.sin(x[0]) * y_noise + x[1]
        y_new = -np.sin(x[0]) * x_noise + np.cos(x[0]) * y_noise + x[2]
        my_cost = my_cost + find_closest(x_new, y_new)
    return my_cost

In [ ]:
def find_error(osm1, osm2):
    masks = np.zeros(osm2.shape[0])
    error = 0
    for i in range(osm1.shape[0]):
        closest = 10000.0
        for j in range(osm2.shape[0]):
            if masks[j] == 0:
                dist = np.sqrt((osm1[i,0] - osm2[j,0])**2 + (osm1[i,1] - osm2[j,1])**2)
                if dist == 0:
                    error = error + dist
                    masks[j] = 1
                    break
                if dist < closest:
                    closest = dist
                    indi = j
        error = error + closest
        masks[indi] = 1
    return error
                    

In [ ]:
for scan_idx in range(3400):
    scan = full_scan.iloc[scan_idx]['scan_utm']
    road_mask = full_scan.iloc[scan_idx]['is_road_truth']
    road_mask = np.where(road_mask)
    xmin = min(scan['x'])
    xmax = max(scan['x'])
    ymin = min(scan['y'])
    ymax = max(scan['y'])
    # print(cloud.iloc[0]['is_road_truth'])
    # print(xmin, xmax, ymin, ymax)
    # plt.plot(scan['x'], scan['y'], '.', ms=1)

    # plot_coords = lambda obj: plt.plot(obj.xy[0],obj.xy[1], 'k')
    # plot_coords(devens_map.exterior)
    # [plot_coords(x) for x in devens_map.interiors]
    # plt.plot((xmin+xmax)/2,(ymin+ymax)/2,'gx')

    # Extract OSM nodes that are within the distance threshold
    osm_thresh = 20.
    xinds = np.where(np.logical_and(osm_nodes[:,0] > xmin, osm_nodes[:,0] < xmax))
    yinds = np.where(np.logical_and(osm_nodes[:,1] > ymin, osm_nodes[:,1] < ymax))
    osm_inds = np.intersect1d(xinds, yinds)

    xinds_gt = np.where(np.logical_and(osm_nodes_gt[:,0] > xmin, osm_nodes_gt[:,0] < xmax))
    yinds_gt = np.where(np.logical_and(osm_nodes_gt[:,1] > ymin, osm_nodes_gt[:,1] < ymax))
    osm_inds_gt = np.intersect1d(xinds_gt, yinds_gt)



    # Active set of OSM nodes
    osm_nodes_active = osm_nodes[osm_inds]
    osm_nodes_gt_active = osm_nodes_gt[osm_inds_gt]

    # Extract LiDAR scan points that correspond to road
    scan_active = scan[road_mask]
    # print(scan_active, scan_active.shape)
    scan_active = np.asarray([[item[0] for item in scan_active], [item[1] for item in scan_active]]).T
    x0 = np.array([0.1, 0.8, 1.2])
    res = minimize(cost_func_2d, x0, method='nelder-mead', options={'xtol':1e-8, 'disp':True})
    #res_bounded = minimize(cost_func_2d, x0, method='trust-constr', options={'verbose':1}, bounds=bounds)
    #print(res.x)
    osm_trans = transform_devens_2d(res.x)
    plt.scatter(devens_trans[:,0], devens_trans[:,1])
    plt.show()
    plt.savefig('/home/sai/TopometricRegistration/devensData/devens'+str(scan_idx)+'.png')
    plt.scatter(road_pts[:,0], road_pts[:,1])
    plt.show()
    plt.savefig('/home/sai/TopometricRegistration/devensData/road'+str(scan_idx)+'.png')
    ori_error = find_error(osm_nodes_active, osm_nodes_gt_active)
    opti_error = find_error(osm_trans, osm_nodes_gt_active)
    print(ori_error, opti_error)